# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [47]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings

warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [48]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,quelimane,-17.8786,36.8883,26.97,65,40,3.09,MZ,2023-03-29 15:37:03
1,1,saskylakh,71.9167,114.0833,-26.57,100,96,2.13,RU,2023-03-29 15:39:07
2,2,avarua,-21.2078,-159.7750,25.03,94,20,4.63,CK,2023-03-29 15:37:03
3,3,saint anthony,45.0205,-93.2180,-6.22,59,0,3.09,US,2023-03-29 15:37:04
4,4,east london,-33.0153,27.9116,20.52,52,0,7.20,ZA,2023-03-29 15:39:08


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [50]:
%%capture --no-display

# Configure the map plot
vacation_cities_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "City",
    frame_width = 700,
    frame_height = 500
)

# Display the map
vacation_cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [51]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_df = city_data_df[(city_data_df["Max Temp"] > 15) & (city_data_df["Max Temp"] < 30)]

# Drop any rows with null values
ideal_temp_df_clean = ideal_temp_df.dropna()

# Display sample data
ideal_temp_df_clean.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,quelimane,-17.8786,36.8883,26.97,65,40,3.09,MZ,2023-03-29 15:37:03
2,2,avarua,-21.2078,-159.7750,25.03,94,20,4.63,CK,2023-03-29 15:37:03
4,4,east london,-33.0153,27.9116,20.52,52,0,7.20,ZA,2023-03-29 15:39:08
7,7,port elizabeth,-33.9180,25.5701,19.50,61,0,8.75,ZA,2023-03-29 15:39:09
8,8,marzuq,14.4000,46.4667,24.53,62,84,1.59,YE,2023-03-29 15:39:10


### Step 3: Create a new DataFrame called `hotel_df`.

In [52]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_temp_df_clean[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
hotel_df = hotel_df.reset_index()
del hotel_df["index"]

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,quelimane,MZ,-17.8786,36.8883,65,
1,avarua,CK,-21.2078,-159.7750,94,
2,east london,ZA,-33.0153,27.9116,52,
3,port elizabeth,ZA,-33.9180,25.5701,61,
4,marzuq,YE,14.4000,46.4667,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
category = "accommodation.hotel"
radius = 10000


params = {
    "categories": category,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [46]:
%%capture --no-display

# Configure the map plot
vacation_cities_map_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "CartoDark",
    color = "City",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
vacation_cities_map_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)